In [8]:
!docker-compose up -d

Creating network "clickhouse_default" with the default driver
Creating clickhouse ... 

Creating clickhouse ... done



In [3]:
!docker-compose down

Stopping clickhouse ... 

Stopping clickhouse ... done

Removing clickhouse ... 

Removing clickhouse ... done

Removing network clickhouse_default


In [6]:
from clickhouse_driver import Client

client = Client(host="localhost") 

In [9]:
client.execute("CREATE DATABASE IF NOT EXISTS analyze")

[]

In [10]:
client.execute("DROP TABLE IF EXISTS analyze.views")

[]

In [13]:
create_sql = """
CREATE TABLE IF NOT EXISTS analyzeviews (
            id String,
            user_id String,
            movie_id String,
            timestamp_movie Int64,
            time Int64
            ) Engine=MergeTree() ORDER BY id
"""
client.execute(create_sql)

[]

In [14]:
client.execute("select count(1) from analyze.views")

[(0,)]

In [33]:
import uuid
from random import randint
from faker import Faker


def generate_random_data(num_rows_per_chunk: int, iteration: int):
    chunk = []
    fake = Faker()
#     event_time = fake.date_time_between()
    for i in range(iteration*num_rows_per_chunk):
        timestamp_movie = randint(1, 100)
        time = randint(1000000000, 10000000000)
        chunk.append((str(uuid.uuid4()), str(uuid.uuid4()), str(uuid.uuid4()), timestamp_movie, time))

    yield chunk

In [35]:
import time
import statistics

class profile_code():
    def __init__(self):
        self.start = time.time()
        self.all_durations = []
        
    def setup_start_time(self):
        self.start = time.time()

    def checkpoint(self):
        end_time = time.time()
        self.all_durations.append(end_time - self.start)
        self.start = end_time

    def __enter__(self):
        return self

    def __exit__(self, type, value, traceback):
        print(f"Median - {statistics.median(self.all_durations)}")
        avg = sum(self.all_durations) / len(self.all_durations)
        print(f"Average - {avg}")
        print(f"Summary - {sum(self.all_durations)}")

In [36]:
# Тестирование вставки по 500 записей за раз
with profile_code() as profiler:
    for chunk in generate_random_data(500, 2000):
        profiler.setup_start_time()
        client.execute(
                    f"INSERT INTO analyze.views (id, user_id, movie_id, timestamp_movie, time) VALUES",
                    chunk,
                )
        profiler.checkpoint()

Median - 8.778651237487793
Average - 8.778651237487793
Summary - 8.778651237487793


In [37]:
# Тестирование одной вставки за раз
with profile_code() as profiler:
    for chunk in generate_random_data(1, 2000):
        profiler.setup_start_time()
        client.execute(
                    f"INSERT INTO analyze.views (id, user_id, movie_id, timestamp_movie, time) VALUES",
                    chunk,
                )
        profiler.checkpoint()

Median - 0.055718421936035156
Average - 0.055718421936035156
Summary - 0.055718421936035156


In [38]:
num_uuids = 100
uuids = [str(u[0]) for u in client.execute(f"SELECT user_id FROM analyze.views LIMIT {num_uuids}")]

In [40]:
# Testing SELECT statments
with profile_code() as profiler:
    for u in uuids:
        profiler.setup_start_time()
        client.execute(f"SELECT * FROM analyze.views where user_id='{u}'")
        profiler.checkpoint()

Median - 0.022065281867980957
Average - 0.02478811264038086
Summary - 2.478811264038086
